In [86]:
# R1.2 修复不存在同频邻区或异频邻区的问题
%%time
import numpy as np
import pandas as pd
import os
os.chdir('E:\cstz_fre')
try:
    tplq=pd.read_csv('查询EUTRAN同频邻区关系.csv',encoding="gbk",usecols=['NAME','本地小区标识','基站标识','小区标识'],dtype={'本地小区标识':str,'基站标识':str,'小区标识':str})
except FileNotFoundError :
    tplq=pd.DataFrame(columns=['NAME','本地小区标识','基站标识','小区标识'])
try:
    yplq=pd.read_csv('查询EUTRAN异频邻区关系.csv',encoding="gbk",usecols=['NAME','本地小区标识','基站标识','小区标识'],dtype={'本地小区标识':str,'基站标识':str,'小区标识':str})
except FileNotFoundError :
    yplq=pd.DataFrame(columns=['NAME','本地小区标识','基站标识','小区标识'])
cell_info=pd.read_excel('小区信息.xlsx',sheet_name='小区基础信息',dtype={'下行频点':str})
fre=pd.read_excel('fre.xlsx',dtype={'上行频点_NEW':str})
wb=pd.read_csv('查询EUTRAN外部小区.csv',encoding="gbk",usecols=['NAME','基站标识','小区标识','下行频点'],dtype={'基站标识':str,'小区标识':str,'下行频点':str})

Wall time: 43.7 s


In [88]:
tplq['NAME_本地小区标识']=tplq['NAME']+"_"+tplq['本地小区标识']

In [89]:
tplq['基站标识_小区标识']=tplq['基站标识']+"_"+tplq['小区标识']

In [90]:
yplq['NAME_本地小区标识']=yplq['NAME']+"_"+yplq['本地小区标识']

In [91]:
yplq['基站标识_小区标识']=yplq['基站标识']+"_"+yplq['小区标识']

In [93]:
lq_s=pd.concat([tplq,yplq])

In [94]:
# 定义删除邻区和调整频点及核查的函数
def stplq(a,b,c,d):
    return f"""RMV EUTRANINTRAFREQNCELL:LOCALCELLID={b},ENODEBID={c},CELLID={d};{{{a}}}"""
def syplq(e,f,g,h):
    return f"""RMV EUTRANINTERFREQNCELL:LOCALCELLID={f},ENODEBID={g},CELLID={h};{{{e}}}"""
def syppd(cc,dd,ee):
    return f"""RMV EUTRANINTERNFREQ:LOCALCELLID={cc},DLEARFCN={dd};{{{ee}}}"""
def tzpd(i,j,k,l,m,n,aa,bb):
    if aa=="TDD":
        return f"""MOD CELL:LOCALCELLID={i},FREQBAND={j},DLEARFCN={k},ULBANDWIDTH={l},DLBANDWIDTH={m};{{{n}}}"""
    else:
        return f"""MOD CELL:LOCALCELLID={i},FREQBAND={j},ULEARFCNCFGIND=CFG,ULEARFCN={bb},DLEARFCN={k},ULBANDWIDTH={l},DLBANDWIDTH={m};{{{n}}}"""
def heca1(ff): 
    return f"""DSP CELL:;{{{ff}}}"""
def heca2(ff):
    return f"""LST CELL:;{{{ff}}}"""
def heca3(ff):
    return f"""LST ALMAF:;{{{ff}}}"""
lq_s['mml1']=lq_s.apply(lambda x : stplq(x['NAME'],x['本地小区标识'],x['基站标识'],x['小区标识']),axis=1)  
lq_s['mml2']=lq_s.apply(lambda y : syplq(y['NAME'],y['本地小区标识'],y['基站标识'],y['小区标识']),axis=1)
fre['mml1']=fre.apply(lambda z : syppd(z['本地小区标识'],z['下行频点_NEW'],z['NAME']),axis=1)
fre['mml2']=fre.apply(lambda x1 : tzpd(x1['本地小区标识'],x1['频带_NEW'],x1['下行频点_NEW'],x1['上行带宽_NEW'],x1['下行带宽_NEW'],x1['NAME'],x1['小区双工模式'],x1['上行频点_NEW']),axis=1)
fre['mml3']=fre.apply(lambda x1_1 : heca1(x1_1['NAME']),axis=1)
fre['mml4']=fre.apply(lambda x1_1 : heca2(x1_1['NAME']),axis=1)
fre['mml5']=fre.apply(lambda x1_1 : heca3(x1_1['NAME']),axis=1)   

In [95]:
# 修改外部函数
wb['基站标识_小区标识']=wb['基站标识']+"_"+wb['小区标识']
wb=pd.merge(wb,fre,how='left',left_on='基站标识_小区标识',right_on='eNodeB标识_小区标识')
wb=wb[['NAME_x','基站标识','小区标识_x','下行频点_NEW']]
def tzwb(o,p,q,r):
    return f"""MOD EUTRANEXTERNALCELL:MCC="460",MNC="00",ENODEBID={p},CELLID={q},DLEARFCN={r};{{{o}}}"""
wb['mml']=wb.apply(lambda y1 : tzwb(y1['NAME_x'],y1['基站标识'],y1['小区标识_x'],y1['下行频点_NEW']),axis=1)

In [96]:
lq=pd.concat([tplq,yplq])

In [60]:
lq=lq[['NAME', '本地小区标识', '基站标识', '小区标识', 'NAME_本地小区标识', '基站标识_小区标识']]

In [61]:
# 生成新的小区信息
fre_new=fre[['eNodeB标识_小区标识', 'NAME_本地小区标识', '小区名称', '小区激活状态', '本地小区标识','频带_NEW',
       '下行频点_NEW', '小区标识', '物理小区标识', '上下行子帧配比', '特殊子帧配比', '本地跟踪区域标识', '跟踪区域码',
       'NAME', 'eNodeB标识', 'UE最大允许发射功率(毫瓦分贝)', '参考信号功率(0.1毫瓦分贝)', '根序列索引',
       'ECI', '小区级参考信号端口数', '小区发送和接收模式', '下行带宽_NEW', '小区双工模式', '多RRU共小区指示']]
fre_new.rename(columns={'频带_NEW':'频带','下行频点_NEW':'下行频点', '下行带宽_NEW': '下行带宽'},inplace=True)
cell_info_new=cell_info[['eNodeB标识_小区标识', 'NAME_本地小区标识', '小区名称', '小区激活状态', '本地小区标识', '频带', '下行频点', '小区标识', '物理小区标识', '上下行子帧配比', '特殊子帧配比', '本地跟踪区域标识', '跟踪区域码','NAME', 'eNodeB标识', 'UE最大允许发射功率(毫瓦分贝)', '参考信号功率(0.1毫瓦分贝)', '根序列索引','ECI', '小区级参考信号端口数', '小区发送和接收模式', '下行带宽', '小区双工模式', '多RRU共小区指示']]
cell_info_new=pd.concat([fre_new,cell_info_new])
cell_info_new.replace({'CELL_BW_N100':'20M','CELL_BW_N50':'10M','CELL_BW_N25':'5M','CELL_BW_N75':'15M'},inplace=True)
cell_info_new.drop_duplicates(subset=['eNodeB标识_小区标识'],keep='first',inplace=True)

c:\program files (x86)\python37-32\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [62]:
# 正向邻区信息
lq_z=pd.merge(lq,cell_info_new,how='left',left_on='NAME_本地小区标识',right_on='NAME_本地小区标识')
lq_z=lq_z[['小区名称','本地小区标识_y','eNodeB标识','下行频点','频带']]
lq_z.rename(columns={'小区名称':'本地小区名称','本地小区标识_y':'本小区LOCALCELLID','eNodeB标识':'本小区eNODEBID','下行频点':'本小区频点','频带':'本小区频带'},inplace=True)

In [63]:
# 反向邻区信息
lq_f=pd.merge(lq,cell_info_new,how='left',left_on='基站标识_小区标识',right_on='eNodeB标识_小区标识')
lq_f=lq_f[['小区名称','小区标识_y','eNodeB标识','下行频点','频带','物理小区标识','跟踪区域码']]
lq_f.rename(columns={'小区名称':'邻区小区名称','小区标识_y':'邻区CELLID','eNodeB标识':'邻区eNODEBID','下行频点':'邻区频点','频带':'邻区频带','物理小区标识':'邻区PCI','跟踪区域码':'邻区跟踪区域码'},inplace=True)

In [64]:
# 新的邻区信息
lq_new=pd.concat([lq_z,lq_f],axis=1)

In [65]:
# 删除邻区空值
lq_new=lq_new.dropna()

In [66]:
lq_new['邻区CELLID']=lq_new['邻区CELLID'].astype(int)
lq_new['邻区eNODEBID']=lq_new['邻区eNODEBID'].astype(int)
lq_new['邻区频带']=lq_new['邻区频带'].astype(int)
lq_new['邻区PCI']=lq_new['邻区PCI'].astype(int)
lq_new['邻区跟踪区域码']=lq_new['邻区跟踪区域码'].astype(int)
lq_new['邻区频点']=lq_new['邻区频点'].astype(int)
lq_new['本小区频点']=lq_new['本小区频点'].astype(int)
lq_new['本小区频带']=lq_new['本小区频带'].astype(int)
# lq_new=lq_new.astype(str)

In [67]:
lq_new=lq_new[['本地小区名称', '邻区小区名称','本小区LOCALCELLID', '本小区eNODEBID', '本小区频点', '本小区频带', '邻区CELLID', '邻区eNODEBID', '邻区频点', '邻区频带', '邻区PCI', '邻区跟踪区域码']]

In [68]:
# 定义切换事件函数
def ypsj(a1,b1,c1,d1):
    if (a1==38950 or  a1==39148) and (c1==39148 or c1==38950):
        return "EventA3"
    elif (a1==37900 or  a1==38098 or a1==40936 or a1==41134 or a1==41332) and (c1==37900 or  c1==38098 or c1==40936 or c1==41134 or c1==41332):
        return "EventA3"
    elif (b1==39 or b1==3) and (d1==3 or d1==39):
        return "EventA3"
    elif (a1==38400 or a1==38496 )and (c1==39292 or c1==39148):
        return "EventA3"
    elif (b1==41 or b1==38 or b1==34) and (d1==40 or d1==3 or d1==39):
        return "EventA4"
    elif (b1==3 or b1==40 )and (d1==3 or d1==40):
        return "EventA4"
    elif (b1==3)and (d1==34 or  d1==38 or d1==41):
        return "EventA4"
    elif (b1==39)and (d1==40 or d1==38 or d1==41 or d1==34):
        return "EventA4"
    elif (b1==8) and (d1==3 or d1==34 or d1==38 or d1==39 or d1==40 or d1==41 ):
        return "EventA4"
    elif (b1==40) and (d1==8 or d1==34 or d1==38 or d1==39 or d1==41):
        return "EventA5"
    elif (b1==3 or b1==34 or b1==38 or b1==39 or b1==41) and (d1==8):
        return "EventA5"
    elif (a1==c1):
        return "EventA3"
    else:
        return "EventA4"
lq_new['异频切换事件类型']=lq_new.apply( lambda z1: ypsj( z1['本小区频点'] , z1['本小区频带'],z1['邻区频点'],z1['邻区频带']),axis=1)       

In [69]:
# 定义异频频点函数
def yppddk(e1):
    if e1==36275 :
        return "MBW75"
    elif (e1==38352 or e1==39292 or e1==38544 or e1==1400 or e1==3590):
        return "MBW50"
    elif ( e1==3682 ):
        return "MBW25"
    else:
        return "MBW100"
lq_new['异频相邻频点带宽']=lq_new.apply( lambda x2 : yppddk (x2['邻区频点']),axis=1)

In [70]:
# 定义优先级函数
def priority (f1,h1,i1,j1):
    if  (h1==3 or h1==34 or h1==38 or h1==39 or h1==40 or h1==41) and j1==8 :
        return 2
    elif (h1==38 or h1==41 or h1==34) and j1==3:
        return 3
    elif (h1==38 or h1==41 or h1==34 or h1==39 or h1==3 or h1==8 or h1==40) and j1==3:
        return 3
    elif (h1==38 or h1==41 or h1==34 or h1==39 or h1==8 or h1==40) and (j1==38 or j1==41 or j1==34):
        return 4
    elif (h1==39 or h1==40 or h1==8)and j1==3:
        return 4
    elif (h1==38 or h1==41 or h1==34 or h1==39 or h1==3 or h1==8 or h1==40) and (j1==40):
        return 5
    elif (h1==3)and (j1==38  or  j1==41 or j1==34):
        return 5
    else:
        return 3  
lq_new['重选优先级']=lq_new.apply( lambda y2: priority (y2['本小区频点'],y2['本小区频带'],y2['邻区频点'],y2['邻区频带']),axis=1)

In [71]:
# 定义高优先级函数
def xhigh(k1,l1):
    if(k1==38 or  k1==41 or k1==34 or k1==3 or k1==39) and (l1==40):
        return 14
    elif (k1==39) and (l1==3):
        return 14
    elif (k1==39 or k1==8) and (l1==38 or l1==41 or l1==34):
        return 15
    elif (k1==8) and (l1==40):
        return 15
    elif (k1==8) and (l1==3):
        return 15
    elif (k1==8) and (l1==39):
        return 15
    elif (k1==3) and (l1==38 or l1==41 or l1==34):
        return 18
    else:
        return 16
lq_new['异频频点高优先级重选门限']=lq_new.apply( lambda z2: xhigh (z2['本小区频带'],z2['邻区频带']),axis=1)   

In [72]:
# 定义低优先级函数
def xlow(m1,n1):
    if(m1==3 or m1==34 or m1==38 or m1==39 or m1==40 or m1==41) and (n1==8):
        return 9
    else:
        return 11
lq_new['异频频点低优先级重选门限']=lq_new.apply( lambda x3: xlow (x3['本小区频带'],x3['邻区频带']),axis=1)

In [73]:
# 定义连接态频率偏执函数
def pz(o1,p1,q1,r1):
    if(o1==38400  or  o1==38496 ) and (q1==38544 or q1==38352):
        return "dB-2"
    elif(p1==3)and(r1==8):
        return "dB4"
    elif (p1==39)and(r1==8):
        return "dB6"
    else:
        return "dB0"
lq_new['连接态频率偏执']=lq_new.apply( lambda y3: pz (y3['本小区频点'],y3['本小区频带'],y3['邻区频点'],y3['邻区频带']),axis=1)    

In [74]:
lq_new=pd.merge(lq_new,cell_info_new,how='left',left_on='本地小区名称',right_on='小区名称')

In [75]:
lq_new=lq_new[['本地小区名称', '邻区小区名称', '本小区LOCALCELLID', '本小区eNODEBID', '本小区频点', '本小区频带','邻区CELLID', '邻区eNODEBID', '邻区频点', '邻区频带', '邻区PCI', '邻区跟踪区域码','异频切换事件类型', '异频相邻频点带宽', '重选优先级', '异频频点高优先级重选门限', '异频频点低优先级重选门限','连接态频率偏执','NAME']]

In [76]:
# 去除附带小区站名的空值邻区
lq_new=lq_new.dropna()

In [77]:
# 定义添加异频频点函数
def tjyppd(a3,b3,c3,d3,e3,f3,g3,h3,i3,j3):
    if a3==c3:
          return None
    else:
        return f"""ADD EUTRANINTERNFREQ:LOCALCELLID={b3},DLEARFCN={c3},ULEARFCNCFGIND=NOT_CFG,CELLRESELPRIORITYCFGIND=CFG,CELLRESELPRIORITY={d3},SPEEDDEPENDSPCFGIND=NOT_CFG,MEASBANDWIDTH={e3},THRESHXHIGH={f3},THRESHXLOW={g3},QRXLEVMIN=-64,PMAXCFGIND=NOT_CFG,PRESENCEANTENNAPORT1=BOOLEAN_TRUE,INTERFREQHOEVENTTYPE={h3},QQUALMINCFGIND=NOT_CFG,CONNFREQPRIORITY=5,QOFFSETFREQCONN={i3};{{{j3}}}"""
lq_new['mml1']=lq_new.apply( lambda z3: tjyppd(z3['本小区频点'],z3['本小区LOCALCELLID'],z3['邻区频点'],z3['重选优先级'],z3['异频相邻频点带宽'],z3['异频频点高优先级重选门限'],z3['异频频点低优先级重选门限'],z3['异频切换事件类型'],z3['连接态频率偏执'],z3['NAME']),axis=1)

In [78]:
# 定义添加外部定义函数
def tjwb(a4,b4,c4,d4,e4,f4,g4,h4):
    if a4==b4:
        return None
    else:
        return f"""ADD EUTRANEXTERNALCELL:MCC="460",MNC="00",ENODEBID={b4},CELLID={c4},DLEARFCN={d4},ULEARFCNCFGIND=NOT_CFG,PHYCELLID={e4},TAC={f4},CELLNAME="{g4}";{{{h4}}}"""
lq_new['mml2']=lq_new.apply( lambda x4: tjwb(x4['本小区eNODEBID'],x4['邻区eNODEBID'],x4['邻区CELLID'],x4['邻区频点'],x4['邻区PCI'],x4['邻区跟踪区域码'],x4['邻区小区名称'],x4['NAME']),axis=1)

In [79]:
# 定义添加邻区函数
def tjlq(a5,b5,c5,d5,e5,f5,g5,h5):
    if a5==b5:
        return f"""ADD EUTRANINTRAFREQNCELL:LOCALCELLID={c5},MCC="460",MNC="00",ENODEBID={d5},CELLID={e5},CELLINDIVIDUALOFFSET=dB0,LOCALCELLNAME="{f5}",NEIGHBOURCELLNAME="{g5}";{{{h5}}}"""
    else:
        return f"""ADD EUTRANINTERFREQNCELL:LOCALCELLID={c5},MCC="460",MNC="00",ENODEBID={d5},CELLID={e5},CELLINDIVIDUALOFFSET=dB0,LOCALCELLNAME="{f5}",NEIGHBOURCELLNAME="{g5}";{{{h5}}}"""
                     

lq_new['mml3']=lq_new.apply( lambda y4: tjlq(y4['本小区频点'],y4['邻区频点'],y4['本小区LOCALCELLID'],y4['邻区eNODEBID'],y4['邻区CELLID'],y4['本地小区名称'],y4['邻区小区名称'],y4['NAME']),axis=1)

In [80]:
# 导出删除邻区脚本
syppd=fre[['mml1']]
syppd.rename(columns={'mml1':'mml'},inplace=True)
stplq=lq_s[['mml1']]
stplq.rename(columns={'mml1':'mml'},inplace=True)
syplq=lq_s[['mml2']]
syplq.rename(columns={'mml2':'mml'},inplace=True)

stplq=stplq[['mml']]
syplq=syplq[['mml']]

syppd=syppd[['mml']]
slq=pd.concat([stplq,syplq,syppd])
slq.to_csv('1_slq.txt',index=False,header=False,encoding='gbk',sep='"')

In [81]:
# 导出调整外部脚本
tzpd_1=fre[['mml2']]
tzpd_1.to_csv('2_tzpd.txt',index=False,header=False,encoding='gbk',sep='"')


In [82]:
# 导出核查脚本
heca=fre[['mml3','mml4','mml5']]
heca=heca.unstack(level=-1)
heca=pd.DataFrame(heca)
heca.rename(columns={0:'mml'},inplace=True)
heca.drop_duplicates(inplace=True)
heca.to_csv('3_heca.txt',index=False,header=False,encoding='gbk',sep='"')


In [83]:
# 导出调整外部脚本
tzwb_1=wb[['mml']]
tzwb_1.to_csv('4_tzwb.txt',index=False,header=False,encoding='gbk',sep='?',quoting=3,doublequote=False,escapechar='?')

In [84]:
# 导出添加邻区脚本
tjlq=lq_new[['mml1','mml2','mml3']]
tjlq=tjlq.unstack(level=-1)
tjlq=pd.DataFrame(tjlq)
tjlq.rename(columns={0:'mml'},inplace=True)
tjlq.drop_duplicates(inplace=True)
tjlq.dropna(subset=['mml'],inplace=True)
# 删除空值
while None in tjlq:
    tjlq.remove(None)
tjlq.to_csv('5_tjlq.txt',index=False,header=False,encoding='gbk',sep='?',quoting=3,doublequote=False,escapechar='?')

In [99]:
# 导出excel格式的脚本
writer = pd.ExcelWriter("1-5频点调整脚本.xlsx")
# 设置不导出索引和将不同的Dateframe 写入多个excel的sheet中
lq_s.to_excel(writer,'第1步删除邻区',index=None)
fre.to_excel(writer,'第2步删异频频点、修改频点',index=None)
wb.to_excel(writer,'第3步调整外部',index=None)
lq_new.to_excel(writer,'第4步添加邻区',index=None)
writer.save()
print("异频脚本已导出成功")

异频脚本已导出成功
